In [1]:
import torch

from flair.data import Sentence
from flair.embeddings import DocumentPoolEmbeddings, FlairEmbeddings, BertEmbeddings

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

## 1. Load flair, BERT cased, BERT uncased embeddings

In [6]:
# [[flair_eu, bert_cased_eu, bert_uncased eu], [flair_en, bert_cased_en, bert_uncased en]]
embeddings_all = [[], []]
embedding_names = ["flair", "bert_cased", "bert_uncased"]

In [7]:
# load flair embeddings
embeddings_all[0].append(DocumentPoolEmbeddings([FlairEmbeddings('eu-forward'), FlairEmbeddings('eu-backward')]))
embeddings_all[1].append(DocumentPoolEmbeddings([FlairEmbeddings('mix-forward'), FlairEmbeddings('mix-backward')]))

In [8]:
# load BERT embeddings

# See BERT paper, section 5.3 and table 7
bert_layers = '-1,-2,-3,-4'
bert_type = 'base' # 'large'

# BERT cased
embeddings_all[0].append(DocumentPoolEmbeddings([BertEmbeddings('bert-base-multilingual-cased', layers=bert_layers)]))
embeddings_all[1].append(DocumentPoolEmbeddings([BertEmbeddings('bert-'+bert_type+'-cased', layers=bert_layers)]))

# BERT uncased
embeddings_all[0].append(DocumentPoolEmbeddings([BertEmbeddings('bert-base-multilingual-uncased', layers=bert_layers)]))
embeddings_all[1].append(DocumentPoolEmbeddings([BertEmbeddings('bert-'+bert_type+'-uncased', layers=bert_layers)]))

2019-07-05 16:59:59,104 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
2019-07-05 17:00:55,218 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


## 2. Read file and extract sentences

In [45]:
test_eu = "goldstandard_eu_lexicover.tsv"
test_en = "goldstandard_en_lexicover.tsv"

def get_gold_sentences(filename):
    gold_sentences = []
    with open(filename, 'rt') as f_p:
        for line in f_p:
            if line.startswith('"origin"'): # header
                continue
            
            if not line:
                continue
            
            line = line.rstrip()
            line = line.replace('"', '')
            splitted = line.split('\t')
            gold = splitted[0]
            sim_sentences = splitted[1:11]
            
            if gold:
                gold_sentences_simple = {}
                gold_sentences_simple[gold] = sim_sentences
                gold_sentences.append(gold_sentences_simple)
            
    return gold_sentences

In [58]:
sent_eu = get_gold_sentences(test_eu)
sent_en = get_gold_sentences(test_en)

In [47]:
def initialize_vectors(sent):
    similarities_all = []
    for i in range(len(sent)):
        similarities_all.append([])

    scores_all = []
    for i in range(len(sent)):
        scores_all.append([])
        
    return similarities_all, scores_all

In [59]:
similarities_all_eu, scores_all_eu = initialize_vectors(sent_eu)
similarities_all_en, scores_all_en = initialize_vectors(sent_en)

## 3. Calculate similarities

In [43]:
def calculate_similarities(gold, sim_sentences, embeddings):
    
    similarities = []
    query = gold

    q = Sentence(query)
    embeddings.embed(q)
    score = 0
    
    for i in range(len(sim_sentences)):
        
        s = Sentence(sim_sentences[i])
        embeddings.embed(s)

        assert q.embedding.shape == s.embedding.shape
        
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        prox = cos(q.embedding, s.embedding)
        similarities.append(round(prox.item(), 4))
        
        if i > 0 and similarities[i] <= similarities[i-1]:
            score += 1
        
    return similarities, score

def calculate(gold_sentences, embeddings, similarities_all, scores_all):
    for i in range(len(gold_sentences)):
        
        # obtain gold sentence and similar sentences from global list
        gold = list(gold_sentences[i].keys())[0]
        sim_sentences = gold_sentences[i][gold]
        
        # Calculate similarities for each 'gold' sentence and accumulated score
        similarities, score = calculate_similarities(gold, sim_sentences, embeddings)

        # append current similarity values and score to the global data structures
        scores_all[i].append(score)
        similarities_all[i].append(similarities)
        
    return similarities_all, scores_all

In [60]:
for i in range(len(embeddings_all[0])):
    print(f"Calculating {embedding_names[i]} embeddings for basque")
    similarities_all_eu, scores_all_eu = calculate(sent_eu, embeddings_all[0][i], similarities_all_eu, scores_all_eu)
    print(f"Calculating {embedding_names[i]} embeddings for english")
    similarities_all_en, scores_all_en = calculate(sent_en, embeddings_all[1][i], similarities_all_en, scores_all_en)

Calculating flair embeddings for basque
Calculating flair embeddings for english
Calculating bert_cased embeddings for basque
Calculating bert_cased embeddings for english
Calculating bert_uncased embeddings for basque
Calculating bert_uncased embeddings for english


## 4. Plot similarities

In [61]:
fig = plotly.tools.make_subplots(rows=1, cols=10)

def plot_similarities(sent, similarities_all, scores_all):
    origin = list(sent[i].keys())[0]
    # print origin sentence
    print(origin + '\n')
    for j in range(len(sent[i][origin])):
        # print each similar sentence
        print(f"{j}. {sent[i][origin][j]}")
        for k in range(len(embedding_names)):
            # print each similarity value for each variant
            print(f"\t {embedding_names[k]} similarity: {similarities_all[i][k][j]}")
    # print scores for all variants
    print(f"Scores: " + ", ".join(f"{embed}: {scor}" for embed, scor in zip(embedding_names, scores_all[i])))
    
    # plot similarity heatmap
    trace = go.Heatmap(z=similarities_all[i], y=embedding_names, colorscale='Blues')
    data=[trace]
    fig.append_trace(trace, 1, i+1)
    iplot(data, filename='basic-heatmap' + str(i))

for i in range(len(sent_eu)):
    
    print(f"Sentence #{i}")
    print("Basque")
    plot_similarities(sent_eu, similarities_all_eu, scores_all_eu)
    print("English")
    plot_similarities(sent_en, similarities_all_en, scores_all_en)

This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]     [ (1,4) x4,y4 ]     [ (1,5) x5,y5 ]     [ (1,6) x6,y6 ]     [ (1,7) x7,y7 ]     [ (1,8) x8,y8 ]     [ (1,9) x9,y9 ]     [ (1,10) x10,y10 ]

Sentence #0
Basque
Medikuak gaixoa bazkari batera gonbidatu zuen

0. Medikuak gaixoa afari batera gonbidatu zuen
	 flair similarity: 0.9597
	 bert_cased similarity: 0.9842
	 bert_uncased similarity: 0.9823
1. Medikuak gaixoa bazkari batera gonbidatu du
	 flair similarity: 0.9283
	 bert_cased similarity: 0.9879
	 bert_uncased similarity: 0.978
2. Medikuak gaixoari esan zion, sendatuko zela
	 flair similarity: 0.6528
	 bert_cased similarity: 0.9237
	 bert_uncased similarity: 0.9265
3. Irakasleak ikaslea bazkari batera gonbidatu zuen
	 flair similarity: 0.8803
	 bert_cased similarity: 0.9593
	 bert_uncased similarity: 0.9406
4. Suhiltzaileak zientzialaria bazkari batera gonbidatu zuen
	 flair similarity: 0.8831
	 bert_cased similarity: 0.9216
	 bert_uncas

English
the doctor invited the patient for lunch

0. the patient invited the doctor for lunch
	 flair similarity: 0.9852
	 bert_cased similarity: 0.9965
	 bert_uncased similarity: 0.984
1. the doctor invited the patient for dinner
	 flair similarity: 0.979
	 bert_cased similarity: 0.9943
	 bert_uncased similarity: 0.9781
2. the doctor has invited the patient for lunch
	 flair similarity: 0.9799
	 bert_cased similarity: 0.9846
	 bert_uncased similarity: 0.9366
3. the patient did not invite the doctor for lunch
	 flair similarity: 0.9127
	 bert_cased similarity: 0.9667
	 bert_uncased similarity: 0.8959
4. the doctor examined the patient before lunch
	 flair similarity: 0.9134
	 bert_cased similarity: 0.9579
	 bert_uncased similarity: 0.8866
5. the doctor thought of inviting the patient for lunch
	 flair similarity: 0.9302
	 bert_cased similarity: 0.9763
	 bert_uncased similarity: 0.8961
6. having lunch was not important for the patient and the doctor
	 flair similarity: 0.8328
	 bert_cas

## 5. Calculate total scores

In [41]:
def calculate_total_score(scores_all):
    total_scores = [0] * len(scores_all[0])
    for i in range(len(scores_all)):
        total_scores[0] += scores_all[i][0]
        total_scores[1] += scores_all[i][1]
        total_scores[2] += scores_all[i][2]
    print(f"Total scores: " + ", ".join(f"{embed}: {scor}" for embed, scor in zip(embedding_names, total_scores)))

In [42]:
calculate_total_score(scores_all_eu)
calculate_total_score(scores_all_en)

Total scores: flair: 6, bert_cased: 5, bert_uncased: 4
Total scores: flair: 5, bert_cased: 5, bert_uncased: 5
